In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [28]:
import numpy as np

def solve_linear_system(A, b):
    m, n = A.shape
    AB = np.hstack((A.astype(float), np.expand_dims(b.astype(float), axis=1)))

    for i in range(min(m, n)):
        pivot_row = i
        while pivot_row < m and AB[pivot_row, i] == 0:
            pivot_row += 1

        if pivot_row == m:
            continue

        if pivot_row != i:
            AB[[i, pivot_row]] = AB[[pivot_row, i]]

        pivot = AB[i, i]
        AB[i] /= pivot

        for j in range(i + 1, m):
            AB[j] -= AB[i] * AB[j, i]

    return AB

    return augmented_matrix

def find_basic_solution(augmented_matrix):
    # 找到自由变量的索引
    num_rows, num_cols = augmented_matrix.shape
    free_variable_indices = []
    for i in range(num_rows):
        if np.all(augmented_matrix[i, :-1] == 0):
            free_variable_indices.append(i)

    # 打印自由变量索引
    print("自由变量的索引:", free_variable_indices)

    # 找到基础解系
    basic_solutions = []
    for free_var_index in free_variable_indices:
        solution = np.zeros(num_cols - 1)
        solution[free_var_index] = 1
        for i in range(num_rows):
            if i != free_var_index:
                solution[i] = augmented_matrix[i, -1] - np.dot(augmented_matrix[i, :-1], solution)
        basic_solutions.append(solution)

    return basic_solutions

A = np.array([[1, 0, 1, 0, 0, 0, -1, 0, 0, 0],
              [1, 0, 0, 1, 0, 0, 0, -1, 0, 0],
              [1, 0, 0, 0, 1, 0, 0, 0, -1, 0],
              [1, 0, 0, 0, 0, 1, 0, 0, 0, -1],
              [0, 0, 0, 0, 1, 1, 0, 0, 0    , 0],
              [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]])

b = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# 解线性方程组并找到阶梯矩阵
augmented_matrix = solve_linear_system(A, b)
print("阶梯矩阵:")
print(augmented_matrix)

# 找到基础解析
basic_solutions = find_basic_solution(augmented_matrix)
print("基础解系:")
for solution in basic_solutions:
    print(solution)


阶梯矩阵:
[[ 1.  0.  1.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.]
 [-0. -0.  1. -0.  1. -0. -0.]
 [ 0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
自由变量的索引: [5]
基础解系:
[ 0.  0. -0.  0.  1.  1.]


In [23]:
import numpy as np

def row_echelon_form(A, b):
    m, n = A.shape
    AB = np.hstack((A.astype(float), np.expand_dims(b.astype(float), axis=1)))

    for i in range(min(m, n)):
        pivot_row = i
        while pivot_row < m and AB[pivot_row, i] == 0:
            pivot_row += 1

        if pivot_row == m:
            continue

        if pivot_row != i:
            AB[[i, pivot_row]] = AB[[pivot_row, i]]

        pivot = AB[i, i]
        AB[i] /= pivot

        for j in range(i + 1, m):
            AB[j] -= AB[i] * AB[j, i]

    return AB[:, :-1], AB[:, -1]

A = np.array([[1, 0, 1, 0, 0, 0, -1, 0, 0, 0],
              [1, 0, 0, 1, 0, 0, 0, -1, 0, 0],
              [1, 0, 0, 0, 1, 0, 0, 0, -1, 0],
              [1, 0, 0, 0, 0, 1, 0, 0, 0, -1],

              [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]])

b = np.array([0, 0,  0, 0, 0, 0])

A_rref, b_rref = row_echelon_form(A, b)
print("行阶梯形式矩阵 A:")
print(A_rref)
print("行阶梯形式向量 b:")
print(b_rref)


行阶梯形式矩阵 A:
[[ 1.   0.   1.   0.   0.   0.  -1.   0.   0.   0. ]
 [ 0.   0.  -1.   1.   0.   0.   1.  -1.   0.   0. ]
 [-0.  -0.   1.  -0.  -1.  -0.  -1.  -0.   1.  -0. ]
 [ 0.   0.   0.   1.   1.   0.   1.   0.  -1.   0. ]
 [ 0.   0.   0.   0.   1.   1.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   1.   0.   0.   0.5 -0.5]]
行阶梯形式向量 b:
[ 0.  0. -0.  0.  0.  0.]


In [80]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [(sigma_plus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])
        term2 = reduce(np.kron, [(sigma_minus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term3 = reduce(np.kron, [(sigma_plus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term4 = reduce(np.kron, [(sigma_minus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])
        hamiltonian += term1 @ term2 + term3 @ term4
    return hamiltonian

# [ 0.  0. -0.  1. -0.  0. -1.  1.  0.  0.]
# [ 0.   0.  -1.  -1.   1.  -0.5  1.   0.   1.   0. ]
# [ 0.   0.  -0.   0.  -1.   0.5  1.   0.   0.   1. ]
v = np.array([[ 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 
              [ 0, 0, 0, 0, 1, 0, 1, 0, 1, 0], 
              [ 0, 0, 0, 0, 0, 0.5, 1, 0, 0, 1]])
w = np.array([[ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [ 0, 0, 1, 1, 0, 0.5, 1, 0, 0, 0], 
              [ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])
C = calculate_hamiltonian(w, v)
num = len(v[0])
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

0000000000
None
====
0000000001
None
====
0000000010
None
====
0000000011
None
====
0000000100
None
====
0000000101
None
====
0000000110
None
====
0000000111
None
====
0000001000
0001000100
====
0000001001
0000100000
====
0000001010
0001000110
====
0000001011
0000100010
====
0000001100
None
====
0000001101
0000100100
====
0000001110
None
====
0000001111
0000100110
====
0000010000
None
====
0000010001
None
====
0000010010
None
====
0000010011
None
====
0000010100
None
====
0000010101
None
====
0000010110
None
====
0000010111
None
====
0000011000
0001010100
====
0000011001
0000110000
====
0000011010
0001010110
====
0000011011
0000110010
====
0000011100
None
====
0000011101
0000110100
====
0000011110
None
====
0000011111
0000110110
====
0000100000
0000001001
====
0000100001
None
====
0000100010
0000001011
====
0000100011
0011000001
====
0000100100
0000001101
====
0000100101
None
====
0000100110
0000001111
====
0000100111
0011000101
====
0000101000
0001100100
====
0000101001
0001100101
===

In [ ]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

depth = 5
params = np.ones(depth * 2)

In [ ]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [(sigma_plus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])
        term2 = reduce(np.kron, [(sigma_minus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term3 = reduce(np.kron, [(sigma_plus**w[i][j] if w[i][j] == 1 else np.eye(2)) for j in range(n)])
        term4 = reduce(np.kron, [(sigma_minus**v[i][j] if v[i][j] == 1 else np.eye(2)) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [ ]:
def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

        
Hp = generate_Hp()

In [ ]:
v = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
w = np.array([[0, 1, 0], 
              [0, 0, 1],
              [1, 0, 0]])
# Hd = gnrt_Hd()
Hd = calculate_hamiltonian(v, w)

In [ ]:
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(Hd)
print(eigenvalues)
# 输出结果
# for i in range(len(eigenvalues)):
#   print("\nEigenvalues:")
#   print(f'{eigenvalues[i].real:.4f}')
#   print("Eigenvectors:")
#   print([f'{num:.4f}' for num in eigenvectors[:, i].real])

In [ ]:
from scipy.linalg import expm
def build_circ(params):
  qc = QuantumCircuit(3)
  beta = params[:depth]
  gamma = params[depth:]
  qc.initialize(eigenvectors[:, 1].real, range(3))
  # qc.x(0)
  # qc.x(2)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * Hp), range(3)) # transpile
    qc.unitary(expm(-1j * beta[dp] * Hd), range(3))
  qc.measure_all()
  return qc

In [ ]:
# print(build_circ(params))

        ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
   q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
        │          ││          ││          ││          ││          │»
   q_1: ┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├»
        │          ││          ││          ││          ││          │»
   q_2: ┤2         ├┤2         ├┤2         ├┤2         ├┤2         ├»
        └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
meas: 3/════════════════════════════════════════════════════════════»
                                                                    »
«        ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐ ░ ┌─┐   »
«   q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├─░─┤M├───»
«        │          ││          ││          ││          ││          │ ░ └╥┘┌─┐»
«   q_1: ┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├─░──╫─┤M├»
«        │          ││          ││          ││    

In [ ]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(3):
    C += cst[i] * num[i]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [ ]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [ ]:
test(40, 5, np.full(5 * 2, np.pi/5))

Iteration 10, Result: 0.0
Iteration 20, Result: 0.0
Iteration 30, Result: 0.0
Iteration 40, Result: 0.0
Iteration 50, Result: 0.0


Iteration 60, Result: 0.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.0
       x: [ 6.283e-01  6.283e-01  6.283e-01  6.283e-01  6.283e-01
            6.283e-01  6.283e-01  6.283e-01  6.283e-01  6.283e-01]
    nfev: 61
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
000 100.0% 0


In [ ]:
test(40, 3, np.full(3 * 2, np.pi/3))

Iteration 10, Result: 0.0
Iteration 20, Result: 0.0
Iteration 30, Result: 0.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.0
       x: [ 1.047e+00  1.047e+00  1.047e+00  1.047e+00  1.047e+00
            1.047e+00]
    nfev: 37
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
000 100.0% 0


In [ ]:
# for i in range(1, 20):
#   print(f'depth == {i}')
#   test(20, i, np.ones(2 * i))